## 빅데이터분석기사 작업형 2

비행탑승 경험 만족도 데이터
- 데이터 설명 : 비행탑승 경험 만족도 (satisfaction 컬럼 : ‘neutral or dissatisfied’ or satisfied )
- x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv
- y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv
- x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv
- x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_test.csv
- 데이터 출처 :https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction?select=train.csv (참고, 데이터 수정)

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv")

display(x_train.head())
display(y_train.head())

,ID,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,id
0,0,Female,Loyal Customer,54,Personal Travel,Eco,1068,3,4,3,...,5,5,3,5,3,5,3,47,22.0,NaN
1,2,Male,Loyal Customer,20,Personal Travel,Eco,1546,4,4,4,...,4,3,3,4,4,4,4,5,2.0,NaN
2,3,Male,Loyal Customer,59,Business travel,Business,2962,0,4,0,...,1,1,1,1,5,1,4,54,46.0,NaN
3,4,Male,Loyal Customer,35,Business travel,Eco Plus,106,5,4,4,...,5,2,1,5,4,4,5,130,121.0,NaN
4,5,Female,Loyal Customer,9,Business travel,Business,2917,3,3,3,...,4,4,4,5,4,3,4,0,0.0,NaN


,ID,satisfaction
0,0,neutral or dissatisfied
1,2,neutral or dissatisfied
2,3,satisfied
3,4,satisfied
4,5,satisfied


In [2]:
# 결측치 확인
print(x_train.isna().sum())
print(x_test.isna().sum())

ID                                       0
Gender                                   0
Customer Type                            0
Age                                      0
Type of Travel                           0
Class                                    0
Flight Distance                          0
Inflight wifi service                    0
Departure/Arrival time convenient        0
Ease of Online booking                   0
Gate location                            0
Food and drink                           0
Online boarding                          0
Seat comfort                             0
Inflight entertainment                   0
On-board service                         0
Leg room service                         0
Baggage handling                         0
Checkin service                          0
Inflight service                         0
Cleanliness                              0
Departure Delay in Minutes               0
Arrival Delay in Minutes               256
id         

In [3]:
# 결측치 처리
x_train['Arrival Delay in Minutes'] = x_train['Arrival Delay in Minutes'].fillna(0)
x_train['id'] = x_train['id'].fillna(0)
x_test['Arrival Delay in Minutes'] = x_test['Arrival Delay in Minutes'].fillna(0)

In [4]:
# 불필요한 컬럼 삭제
x_test_ID = x_test['ID']
x_train = x_train.drop(columns = ['ID'], axis = 1)
x_test = x_test.drop(columns = ['ID'], axis = 1)

In [6]:
# 범주형, 수치형 확인
obj = x_train.select_dtypes(include = 'object').columns
num = x_train.select_dtypes(include = 'number').columns
print(obj)
print(num)

Index(['Gender', 'Customer Type', 'Type of Travel', 'Class'], dtype='object')
Index(['Age', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'id'],
      dtype='object')


In [7]:
# 범주형 전처리
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train['Gender'] = encoder.fit_transform(x_train['Gender'])
x_train['Customer Type'] = encoder.fit_transform(x_train['Customer Type'])
x_train['Type of Travel'] = encoder.fit_transform(x_train['Type of Travel'])
x_train['Class'] = encoder.fit_transform(x_train['Class'])

x_test['Gender'] = encoder.fit_transform(x_test['Gender'])
x_test['Customer Type'] = encoder.fit_transform(x_test['Customer Type'])
x_test['Type of Travel'] = encoder.fit_transform(x_test['Type of Travel'])
x_test['Class'] = encoder.fit_transform(x_test['Class'])

In [8]:
# 수치형 전처리
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [9]:
# 데이터 분리
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train['satisfaction'], test_size = 0.3, random_state = 1234)

In [10]:
# 모델링
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = model.predict_proba(X_TEST)[:,1]

In [11]:
# 평가
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

0.9935295193056923


In [12]:
# 최종제출
y_test_pred = model.predict_proba(x_test)[:,1]
result = pd.DataFrame(y_test_pred)
final = pd.concat([x_test_ID, result], axis =1).rename(columns = {0:'satisfaction'})
final.to_csv('123456.csv', index = False)

In [14]:
# 제출확인
check = pd.read_csv('123456.csv')
print(check)

           ID  satisfaction
0           1          0.21
1          16          0.01
2          17          0.00
3          25          0.00
4          27          0.86
...       ...           ...
20776  103895          0.00
20777  103896          0.00
20778  103897          0.99
20779  103900          0.01
20780  103902          0.03

[20781 rows x 2 columns]
